In [1]:
!pip install langchain-teddynote

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15


In [3]:
!pip install langchain_huggingface

In [4]:
import os

from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [5]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



In [6]:
from langchain_teddynote import logging

logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [7]:
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate

os.environ["PPLX_API_KEY"] = userdata.get('perplexity_api_key')


In [8]:
chat = ChatPerplexity(temperature=0,  model="llama-3.1-sonar-small-128k-chat")

In [9]:
prompt_buykorea = ChatPromptTemplate.from_messages([
    ("system", """Analyze the product description. Identify the product type and extract 5 relevant keywords.
                  Output as a single sentence in the format: '[product type], [keyword1], [keyword2], [keyword3], [keyword4].'
                  No adjectives, adverbs, or additional explanations."""),
    ("human", "{input}")
])

chain_buykorea = prompt_buykorea | chat

In [10]:
paragraph = """ASOME Intensive 7 Effect Total Cream – Visible reduction in dark spots in just 4 weeks.

A functional cream for whitening and wrinkle improvement, clinically proven for 7 key benefits.
Formulated for sensitive skin with a mild, vegan formulation to reduce skin irritation.

This lightweight cream absorbs quickly and glides smoothly onto the skin without any stickiness, helping to reveal a soft, refined skin texture.

With a delicate rose scent, it's suitable for all ages and is universally appealing."""

response_buykorea = chain_buykorea.invoke({"input": paragraph})
print(response_buykorea.content)

Whitening cream, dark spots, wrinkle improvement, sensitive skin, vegan formulation.


In [11]:
prompt_bldata = ChatPromptTemplate.from_messages([
    ("system", """Analyze the product name. Identify the product type and extract 5 relevant keywords.
                  Output as a single sentence in the format: '[product type], [keyword1], [keyword2], [keyword3], [keyword4].'
                  No adjectives, adverbs, or additional explanations."""),
    ("human", "{input}")
])

chain_bldata = prompt_bldata | chat

In [12]:
name = """evn btr mkup spf15-wn"""

response_bldata = chain_bldata.invoke({"input": name})
print(response_bldata.content)

Sunscreen, SPF15, Makeup, Skin, Protection.


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings

# HuggingFaceEmbeddings 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [17]:

# 임베딩할 텍스트
text_buykorea = response_buykorea.content
text_bldata = response_bldata.content

# 텍스트 임베딩
embedding_buykorea = embeddings.embed_query(text_buykorea)
embedding_bldata = embeddings.embed_query(text_bldata)

# 결과 출력
print(f"전처리한 바이코리아 데이터 : {text_buykorea}")
print(f"전처리한 BL 데이터 : {text_bldata}")

print(f"코사인 유사도 : {cosine_similarity(np.array(embedding_buykorea).reshape(1,-1), np.array(embedding_bldata).reshape(1,-1))}")

전처리한 바이코리아 데이터 : Whitening cream, dark spots, wrinkle improvement, sensitive skin, vegan formulation.
전처리한 BL 데이터 : Sunscreen, SPF15, Makeup, Skin, Protection.
코사인 유사도 : [[0.4625173]]
